In [ ]:
%pip install -r requirements.txt

In [ ]:
%pip install optuna ultralytics



In [ ]:
result

In [ ]:
from ultralytics import YOLO
import optuna
import torch

In [2]:
model = YOLO("yolo11n.pt")  # oder yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt

In [ ]:
"""model.train(
    data="traffic_data/data_1.yaml",
    epochs=80,
    imgsz=640,
    batch=16,
    #device=auto,  # RTX 2060
    #workers=8,  # Ryzen 7 2700X
    optimizer="auto",
    lr0=0.001,
    patience=20,
    amp=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=5.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    project="traffic_training",
    name="traffic_yolov8n5",
    save=True,
    plots=True
)"""

In [ ]:
model_next_iter = YOLO("traffic_training/traffic_yolov11n_car5/weights/best.pt")  # Load the latest model from training


In [ ]:

model.train(
    data="traffic_data/traffic_data/data_1.yaml",
    epochs=80,
    imgsz=640,
    batch=16,
    device=0,  # RTX 2060
    #workers=8,  # Ryzen 7 2700X
    #optimizer="auto",
    lr0=0.00025,
    patience=10,
    amp=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=5.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    project="traffic_training",
    name="traffic_yolo11n_BESTMODEL",
    save=True,
    plots=True
)

In [ ]:
result_my = model_next_iter.predict("../traffic_data/train/images/03_jpg.rf.4532f2db68433995da09f54e215160f3.jpg")
for result in result_my:
    result.show()

In [ ]:
model = YOLO("../yolov8n.pt") 
result = model.predict("../traffic_data/train/images/03_jpg.rf.4532f2db68433995da09f54e215160f3.jpg")
for result in result:
    result.show()

In [ ]:
result2 = model.predict("../traffic_data/valid/images/Pias--173-_jpg.rf.929b5f53b64db0cb7ccf17fd9aa19b6e.jpg",save=True)

In [ ]:
print(torch.__version__)  # Should show latest version, e.g., "2.7.1+cu118"

In [ ]:
print(torch.cuda.is_available())  # Should return True

In [ ]:
print(torch.cuda.get_device_name(0))  # Should return "NVIDIA GeForce RTX 2060"

In [ ]:
print(torch.version.cuda)  # Should return "11.8"

In [ ]:
def trial_logic(trial):
    model = YOLO("yolo11n.pt")

    model.train(
        data="traffic_data/traffic_data/data_1.yaml",
        epochs=50,
        imgsz=640,
        batch=16,
        optimizer="auto",
        lr0=trial.suggest_loguniform("lr0", 1e-5, 1e-2),
        patience=10,
        amp=True,
        hsv_h=trial.suggest_float("hsv_h", 0.0, 0.05),
        hsv_s=trial.suggest_float("hsv_s", 0.5, 0.9),
        hsv_v=trial.suggest_float("hsv_v", 0.3, 0.7),
        scale=trial.suggest_float("scale", 0.3, 0.6),
        fliplr=0.5,
        mosaic=1.0,
        project="traffic_training",
        name="traffic_yolo11n_optuna2",
        save=True,
        plots=False
    )

    # Access trainer metrics directly
    try:
        return model.trainer.results_dict["metrics/mAP50-95"]
    except Exception:
        raise optuna.TrialPruned()  # if metric not found, prune the trial


In [ ]:
study = optuna.create_study(direction="maximize")  # because you want to maximize mAP
study.optimize(trial_logic, n_trials=10)

In [ ]:
from ultralytics import YOLO
import torch

# Load model
model = YOLO("yolo11n.pt")

# Launch hyperparameter evolution
model.tune(
    data="traffic_data/traffic_data/data_1.yaml",  # your dataset
    epochs=40,
    iterations=10, 
    device=0,
    imgsz=640,
    batch=16,
    optimizer="auto",               # or 'SGD', 'Adam', 'AdamW'
    project="traffic_training",
    name="yolo11n_MAX",
    cache=True                      # cache images for speed
)


Tuner: Initialized Tuner instance with 'tune_dir=traffic_training/yolo11n_MAX10'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/10 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'cutmix': 0.0, 'copy_paste': 0.0}
New https://pypi.org/project/ultralytics/8.3.156 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.153 🚀 Python-3.11.7 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100 80GB PCIe, 81154MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_past

train: Scanning /pfs/data6/home/ds/ds_ds/ds_wi22202/traffic_data/traffic_data/train/labels... 4056 images, 2 backgrounds, 0 corrupt: 100%|██████████| 4056/4056 [00:03<00:00, 1048.42it/s]


train: New cache created: /pfs/data6/home/ds/ds_ds/ds_wi22202/traffic_data/traffic_data/train/labels.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (2.7GB RAM): 100%|██████████| 4056/4056 [00:01<00:00, 2309.71it/s]


val: Fast image access ✅ (ping: 1.9±0.1 ms, read: 9.7±2.0 MB/s, size: 35.2 KB)


...

      35/40      5.25G      1.052     0.7499     0.9916         94        640: 100%|██████████| 254/254 [00:16<00:00, 15.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:00<00:00, 12.85it/s]


                   all        300       2568      0.548      0.457      0.471      0.269

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      36/40      5.25G      1.041     0.7443     0.9882         57        640: 100%|██████████| 254/254 [00:16<00:00, 15.65it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:00<00:00, 12.54it/s]


                   all        300       2568      0.508       0.49      0.458      0.269

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      37/40      5.25G      1.045     0.7324     0.9856        288        640:  37%|███▋      | 94/254 [00:06<00:10, 15.65it/s]

In [2]:
model = YOLO("traffic_training/train37/weights/best.pt")

# Best Parameters Model training

In [ ]:
from ultralytics import YOLO
import torch

model.train(
    data="traffic_data/traffic_data/data_1.yaml",
    epochs=80,
    imgsz=640,
    batch=16,
    device=0,  # RTX 2060
    optimizer="AdamW",
    lr0=0.00955,
    lrf=0.00998,
    momentum=0.97997,
    weight_decay=0.00054,
    warmup_epochs=2.54319,
    warmup_momentum=0.94159,
    box=7.99327,
    cls=0.42895,
    dfl=1.61663,
    hsv_h=0.01156,
    hsv_s=0.75561,
    hsv_v=0.31895,
    degrees=0.0,
    translate=0.10405,
    scale=0.4834,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.40892,
    bgr=0.0,
    mosaic=1.0,
    mixup=0.0,
    cutmix=0.0,
    copy_paste=0.0,
    patience=30,
    amp=True,
    project="traffic_training",
    name="traffic_yolo11n_BESTMODEL",
    save=True,
    plots=True
)


# Best Model Eval

In [2]:
from ultralytics import YOLO
import torch

In [3]:
best_model = YOLO("traffic_training/train37/weights/best.pt")

In [4]:
best_results = best_model.val(data="traffic_data/traffic_data/data_1.yaml")

Ultralytics 8.3.153 🚀 Python-3.11.7 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100 80GB PCIe, 81154MiB)
YOLO11n summary (fused): 100 layers, 2,585,662 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 2.1±0.5 ms, read: 9.2±1.8 MB/s, size: 34.4 KB)


val: Scanning /pfs/data6/home/ds/ds_ds/ds_wi22202/traffic_data/traffic_data/valid/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: /pfs/data6/home/ds/ds_ds/ds_wi22202/traffic_data/traffic_data/valid/images/Pias--359-_PNG.rf.43bcf36efe5cf8c37552d2c45fffea60.jpg: 9 duplicate labels removed
val: /pfs/data6/home/ds/ds_ds/ds_wi22202/traffic_data/traffic_data/valid/images/Pias--360-_PNG.rf.8405b0e44009a9300e0a1100ccf7d5b3.jpg: 7 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:02<00:00,  6.38it/s]


                   all        300       2568      0.645      0.415       0.54      0.303
               bicycle         30         32      0.648      0.312      0.368       0.17
                   bus        220        425      0.774      0.531      0.644      0.419
                   car        239        922      0.769       0.74      0.791      0.519
               minibus          2          2      0.341        0.5      0.497      0.397
               minivan         87        110      0.486      0.436      0.408      0.308
             motorbike        166        335      0.679       0.51      0.562      0.206
                pickup        105        142      0.625      0.247      0.378      0.232
              rickshaw         62        192       0.74      0.609      0.675      0.428
               scooter          1          1          1          0      0.995      0.199
  three wheelers -CNG-        148        252      0.835      0.595      0.702      0.466
                 truc

In [6]:
print(best_results.box.map)  # list of per-class mAP@0.5
print(best_results.box.map50)  # mAP@0.5
print(best_results.box.map75)  # mAP@0.75

0.3026481379597858
0.5403427900006513
0.3000627020975204


In [9]:
print(best_results.box.map75)  

0.3000627020975204


In [8]:
print(best_results.box.map50)  # mAP@0.5

0.5403427900006513


In [ ]:
best_results

In [ ]:
model = YOLO("yolo11n.pt")

# Train with default settings (acts as a fair baseline)
model.train(
    data="traffic_data/traffic_data/data_1.yaml",  # Your dataset config
    epochs=40,
    imgsz=640,
    batch=16,
    device=0,  # or "cuda:0"
    project="traffic_training",
    name="yolo11n_baseline",
    save=True,
    plots=True
)

In [17]:
result =model.val(data="traffic_data/traffic_data/data_1.yaml")

Ultralytics 8.3.153 🚀 Python-3.11.7 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100 80GB PCIe, 81154MiB)
YOLO11n summary (fused): 100 layers, 2,585,662 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.1±0.0 ms, read: 25.3±6.2 MB/s, size: 35.2 KB)


val: Scanning /pfs/data6/home/ds/ds_ds/ds_wi22202/traffic_data/traffic_data/valid/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]

val: /pfs/data6/home/ds/ds_ds/ds_wi22202/traffic_data/traffic_data/valid/images/Pias--359-_PNG.rf.43bcf36efe5cf8c37552d2c45fffea60.jpg: 9 duplicate labels removed
val: /pfs/data6/home/ds/ds_ds/ds_wi22202/traffic_data/traffic_data/valid/images/Pias--360-_PNG.rf.8405b0e44009a9300e0a1100ccf7d5b3.jpg: 7 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:01<00:00, 10.34it/s]


                   all        300       2568      0.618      0.405      0.467      0.294
               bicycle         30         32      0.756      0.312      0.355      0.142
                   bus        220        425      0.812      0.448      0.616      0.392
                   car        239        922      0.776      0.714       0.79      0.517
               minibus          2          2      0.542        0.5      0.496      0.446
               minivan         87        110      0.528      0.366      0.445      0.329
             motorbike        166        335      0.715      0.439      0.515      0.174
                pickup        105        142      0.664      0.254      0.373      0.227
              rickshaw         62        192      0.718      0.635      0.687      0.429
               scooter          1          1          0          0    0.00415   0.000829
  three wheelers -CNG-        148        252      0.794      0.631      0.718      0.471
                 truc

In [18]:
print(result.box.map)  # list of per-class mAP@0.5
print(result.box.map50)  # mAP@0.5
print(result.box.map75)  # mAP@0.75

0.2940217145082796
0.4666291632607672
0.3086789048616195


In [ ]:
from ultralytics import YOLO
import torch

# Load model
model = YOLO("yolo11n.pt")

# Launch hyperparameter evolution
model.tune(
    data="traffic_data/traffic_data/data_1.yaml",  # your dataset
    epochs=40,
    iterations=10, 
    device=0,
    imgsz=640,
    batch=16,
    optimizer="auto",               # or 'SGD', 'Adam', 'AdamW'
    project="traffic_training",
    name="yolo11n_MAX",
    cache=True                      # cache images for speed
)


Tuner: Initialized Tuner instance with 'tune_dir=traffic_training/yolo11n_MAX11'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/10 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'cutmix': 0.0, 'copy_paste': 0.0}
New https://pypi.org/project/ultralytics/8.3.156 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.153 🚀 Python-3.11.7 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100 80GB PCIe, 81154MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_past

train: Scanning /pfs/data6/home/ds/ds_ds/ds_wi22202/traffic_data/traffic_data/train/labels.cache... 4056 images, 2 backgrounds, 0 corrupt: 100%|██████████| 4056/4056 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (2.7GB RAM): 100%|██████████| 4056/4056 [00:01<00:00, 2646.52it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 28.0±9.7 MB/s, size: 35.2 KB)
val: /pfs/data6/home/ds/ds_ds/ds_wi22202/traffic_data/traffic_data/valid/images/Pias--359-_PNG.rf.43bcf36efe5cf8c37552d2c45fffea60.jpg: 9 duplicate labels removed
val: /pfs/data6/home/ds/ds_ds/ds_wi22202/traffic_data/traffic_data/valid/images/Pias--360-_PNG.rf.8405b0e44009a9300e0a1100ccf7d5b3.jpg: 7 duplicate labels removed
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

       8/40      5.04G      1.421      1.412      1.083        101        640: 100%|██████████| 254/254 [00:16<00:00, 15.09it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:00<00:00, 12.17it/s]


                   all        300       2568      0.531      0.362      0.373      0.226

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       9/40      5.04G      1.404      1.352      1.075        220        640: 100%|██████████| 254/254 [00:16<00:00, 15.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:00<00:00, 11.74it/s]


                   all        300       2568      0.531      0.355      0.394       0.24

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/40      5.04G      1.399      1.344      1.076        307        640:  47%|████▋     | 120/254 [00:08<00:08, 15.39it/s]